# COGS 118A - Final Project

# Used Car Price Prediction

## Group members

- James Callahan
- Dima Musa
- Victor Tran
- Xiaonan Fu

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents 
- the solution/what you did
- major results you came up with (mention how results are measured) 

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Background

Cars are an essential part of everyday life for most Americans. In 2022, around 276 million vehicles were registered in the United States <a name="bts"></a>[<sup>[1]</sup>](#btsnote). In 2019, 40 thousand used light vehicles were purchased in comparison to the 1,500 new light vehicles purchased <a name="usedbts"></a>[<sup>[2]</sup>](#usedbtsnote). There is a clear demand for used cars in the United States. With such high demand, predicting the selling price of a used vehicle would be extremely useful to both sellers and buyers.

There have been multiple attempts to predict car prices using machine learning algorthims. In 2014, Sameerchand Pudaruth attempted to predict the price of used cars using multiple linear regression analysis, k-nearest neighbors, naive bayes and decision trees. Pudaruth based their prediction on make, model, volume of cylinder, mileage in kilometers, year of manufacture, and price. They found comparable predicitions from all machine learning models <a name="pudaruth"></a>[<sup>[3]</sup>](#pudaruthnote).

In 2019, Enis Gegic et al. also attempted to predict the price of used cars. They used artifical neural networks, support vector machine, and random forest. They also used cross validation to evaluate their predicitons. The variables used to make predictions were make, model, fuel type, power in kilowats, year of manufacture, miles, leather (yes/no), cruise control (yes/no), and price. Then, prices were classified based on price ranges. They found applying a single machine learning algorthim to be insufficent as "accuracy was less than 50%" (Gegic et al., 118). Thus, and "ensemble of multiple machine learning algorithms has been proposed and this combination of ML methods gains accuracy of 92.38%" (Gegic et al., 118) <a name="enis"></a>[<sup>[4]</sup>](#enisnote).

# Problem Statement

We want to build a model that would be able to accurately predict the sale price of a used car, taking in to account variables  such as make, model, mileage, accidents, and number of prior owners among others. 

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data

#### [US Used Cars Dataset](https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset?select=used_cars_data.csv)

This data set originally has:
- A whopping 66 variables
- 3,000,000 unique values

The 18 variables we have kept are: 
- body_type: Type String. Body Type of the vehicle. Like Convertible, Hatchback, Sedan, etc.
- latitude: Type Float. Latitude from the geolocation of the dealership.
- longitude: Type Float. Longitude from the geolocation of the dealership.
- daysonmarket: Type Integer. Days since the vehicle was first listed on the website.
- engine_displacement: Type Float. The measure of the cylinder volume swept by all of the pistons of a piston engine, excluding the combustion chambers.
- frame_damage: Type Boolean. Whether the vehicle has a damaged frame.
- fuel_type: Type String. Dominant type of fuel ingested by the vehicle.
- has_accidents: Type Boolean. Whether the vin has any accidents registered.
- horsepower: Type Float. Horsepower is the power produced by an engine.
- lisitng_color:  Type String. Dominant color group from the exterior color.
- major_options: list of strings of features.
- make_name: Type string. Manufacturer of car.
- mileage: Type float. Number of miles driven by car.
- model_name: Type string. 
- owner_count: Type float. Number of owners.
- price: Type float. Listed price of car.
- transmission: Type string. Type of transmission.
- wheel_system: Type string. Type of wheel_system.
- year: Type int. Year of manufacture.

We have one-hot encoded:
- make_name
- major_options

We have categorized:
- price

# Proposed Solution

We propose a comparison across multiple algorithms for solving our problem. Potential methods for solving our problem include linear regression, polynomial regression, k-nearest neighbors (KNN), support vector machine (SVM) , and the use of decision trees. Implementing a variety of supervised algorithms would afford us an opportunity to approach the problem from multiple perspectives, and would allow an opportunity to compare the performance of these various methods. For a benchmark, we will compare our results with other models that have been trained on this data. One such project can be found [here](https://www.kaggle.com/code/valchovalev/car-predictor-usa).

SVM has been used in multiple similar studies for its ability to deal with datasets with more dimensions. SVM can make a binary decision and decide in which among the two categories the input belongs to. KNN compares the new data to all the existing records to decide the best match. Decision trees are best for nominal categories. 

--

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# Evaluation Metrics

The performance of our model will be evaluated with standard and repeated k-folds cross validation. Accuracy will be compared across models to determine which is most accurate. Cross validation is where a given data set is split into a K number of sections/folds where each fold is used as a testing set at some point <a name="k"></a>[<sup>[5]</sup>](#knote).

Repeated K-Folds will allow us to test multiple hyperparameter settings and weights, which will allow us to evaluate the results of our early statistical analysis and the correlations predicted therein.

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

The data we will be using is entirely anonymous. A previously considered dataset included the VIN (Vehicle Identification Number) and location of sale, which could have potentially been used to identify current or past owners of the cars listed. We decided to use this new dataset as it contains more useful variables, and does not include potentially compromising information. We also remvoved location of the car for privacy concerns, however, it is possible the location would have an impact on car price. Cars from more wealthy neighborhoods are likely to be more expensive. 


If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="btsnote"></a>1.[^](#bts): [Number of U.S. Aircraft, Vehicles, Vessels, and Other Conveyances](https://www.bts.gov/content/number-us-aircraft-vehicles-vessels-and-other-conveyances)<br> 
<a name="usedbtsnote"></a>2.[^](#usedbts): [New and Used Passenger Car and Light Truck Sales and Leases](https://www.bts.gov/content/new-and-used-passenger-car-sales-and-leases-thousands-vehicles)<br>
<a name="pudaruthnote"></a>3.[^](#pudaruth): Pudaruth, Sameerchand. "Predicting the price of used cars using machine learning techniques." Int. J. Inf. Comput. Technol 4.7 (2014): 753-764. https://www.academia.edu/download/54261672/2014_Predicting_the_Price_of_Used_Cars_using_Machine_Learning_Techniques.pdf <br>
<a name="enisnote"></a>4.[^](#enis): Gegic, Enis, et al. "Car price prediction using machine learning techniques." TEM Journal 8.1 (2019): 113. https://www.temjournal.com/content/81/TEMJournalFebruary2019_113_118.pdf <br>
<a name="knote"></a>5.[^](#k): [K-Fold Cross Validation](https://medium.datadriveninvestor.com/k-fold-cross-validation-6b8518070833)<br>